In [1]:
from B0729038 import *

from B0729038_Optimization_01 import *

In [2]:
%matplotlib qt

In [3]:
def f(s): 
    x, y= s  
    z=  x+y 
    z= -z   
    return z 

def c1(s): 
    x,y= s 
    z= x**2 + y**2 - 1 
    return z 
 

def c2(s): 
    x,y= s 
    z= (x**2 + y**2 -2) 
    z= -z 
    return z 
 

Constraints= [ 
    {'fun':c1, 'type':'ineq'}, 
    {'fun':c2, 'type':'ineq'}, 
 

] 
 

xyBounds= [(0, None),  # xmin, xmax 
           (0, None)   # ymin, ymax 
           ] 
 

import scipy.optimize as sopt 
 

x0= np.random.random(size=2) 
 

opt= sopt.minimize( 
    f, 
    x0, 
    bounds= xyBounds, 
    constraints= Constraints 
) 
 

x0, opt 

 

(array([0.15790802, 0.56764   ]),
      fun: -2.0000000000299507
      jac: array([-1., -1.])
  message: 'Optimization terminated successfully'
     nfev: 25
      nit: 8
     njev: 8
   status: 0
  success: True
        x: array([1.00000005, 0.99999995]))

In [11]:
x,y= sm.symbols('x,y')

# sympy function --> numpy function
z= sm.lambdify([x,y],f([x,y])) 

xx= np.linspace(-10,10,1001)
yy= np.linspace(-10,10,1001)
zz= z(xx,yy)

xm, ym= np.meshgrid(xx,yy)
zm= z(xm,ym)
zz.shape, zm.shape



#
# plot constraints 
#

#  Constraints 
# 如何自動從 c1, c2, c3 撈出來？？
# 需要 sympy.solve()
#
y1= sm.solve(
        c1([x,y]),
        y)[0]
y2= sm.solve(
        c2([x,y]),
        y)[0]
# sympy 解方程式之後，仍然需要numpy

y1
#y1= sm.lambdify(x, y1)
#y2= sm.lambdify(x, y2)

#yy1= y1(xx)
#yy2= y2(xx)




In [5]:
#-----------------------

#
#  平行x軸,y軸 界線 (xyBounds) 可以如下製造出來 
#
[(xmin,xmax),(ymin,ymax)]= xyBounds
           
y_ymin= ymin *np.ones_like(xx)
y_ymax= ymax *np.ones_like(xx)
x_xmin= xmin *np.ones_like(yy)
x_xmax= xmax *np.ones_like(yy)

#
# 坐標軸 (x軸,y軸) 可以如下製造出來 
#
y_0= np.zeros_like(xx)
x_0= np.zeros_like(yy)

ax= pl.axes(xlim=(-10,10),ylim=(-10,10))#, projection='3d')

ax.contour(xm,ym,zm, 100, cmap='rainbow')

ax.plot(xx,yy1,'r',
        xx,yy2,'g',
        xx,yy3,'b',
        linestyle='--'
       )    





#


TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'

In [12]:
# 畫坐標軸 (x軸,y軸) 
#
ax.plot(xx, y_0,'w-',
        x_0,yy, 'w-',
        linewidth= 3,
        alpha= 0.3 
       )

#
#  畫 xyBounds (平行x軸,y軸) 
#
ax.plot(xx,     y_ymin,
        xx,     y_ymax,
        x_xmin, yy, 
        x_xmax, yy, 
        linewidth= 1,
        linestyle= '--',
        color= 'gray'
       )

# 把 符合限制條件的區域標示出來 (塗滿灰色)
# 這裡比較難以全自動，再想想、、、

ax.fill_between(xx, yy3, yy1,                
                where= (yy3 >= yy1)&(yy1>=yy2),
                alpha=0.50, 
                color='gray', 
                interpolate=True
               )
ax.fill_between(xx, yy3, yy2,                
                where= (yy3 >= yy2)&(yy2>=yy1),
                alpha=0.50, 
                color='gray', 
                interpolate=True
               )

for yyy in [yy1,yy2,yy3]:
    
    ax.fill_between(xx, y_ymax, yyy,                 
                    where= (yyy>=y_ymax),
                    alpha= 0.5, 
                    color='white', 
                    interpolate=True
                   )
    ax.fill_between(xx, y_ymin, yyy,                 
                    where= (yyy<=y_ymin),
                    alpha= 0.5, 
                    color='white', 
                    interpolate=True
                   )






In [13]:
# 在這個區域內找最小 f([x,y])
import scipy.optimize as sopt

x0= [0, 0]

opt= sopt.minimize(
    f,
    x0,
    #method= 'SLSQP',
    bounds= xyBounds,
    constraints=     Constraints
)

opt

#
# 畫 最佳點 opt.x 
#

xopt= opt.x[0]
yopt= opt.x[1]
fopt= opt.fun

ax.scatter(
    x= xopt,
    y= yopt, 
    color= 'magenta',
    marker= 's'    
)

ax.text(
    x= xopt, 
    y= yopt,
    s= f'{xopt:.3f},{yopt:.3f}'
)


fLtx=  sm.latex(f([x,y]))
c1Ltx= sm.latex(c1([x,y]))
c2Ltx= sm.latex(c2([x,y]))
c3Ltx= sm.latex(c3([x,y]))

In [9]:
titleStr= f'''
Optimize:      f(x,y)= ${fLtx}$
Subject to:    Constraints and Bounds
opt= [x= {xopt:.3f}, y= {yopt:.3f}; f= {fopt:.3f}]
'''

ax.set_title(titleStr)
ax.set_xlabel('x')
ax.set_ylabel('y')

infoStr= f'''
$Objective: f(x,y)= {fLtx}$
$c_1(x,y)= {c1Ltx} ≥ 0$ 
$c_2(x,y)= {c2Ltx} ≥ 0$ 
$c_3(x,y)= {c3Ltx} ≥ 0$
$x \in [{xmin},{xmax}]$ 
$y \in [{ymin},{ymax}]$
opt= [x= {xopt:.3f}, y= {yopt:.3f}; f= {fopt:.3f}]
'''
ax.text(
    x= -10, 
    y= -10,
    s= infoStr
)

pl.show()